In [1]:
## Compute '3+4'

In [2]:
import tensorflow as tf
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

(<tf.Tensor 'Const:0' shape=() dtype=float32>, <tf.Tensor 'Const_1:0' shape=() dtype=float32>)


In [3]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [4]:
node3 = tf.add(node1, node2)
print("node3: ",node3)
print("sess.run(node3): ",sess.run(node3))


('node3: ', <tf.Tensor 'Add:0' shape=() dtype=float32>)
('sess.run(node3): ', 7.0)


In [5]:
## Compute ‘((3+4)^2)*5’

In [6]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0, tf.float32)
node3 = tf.add(node1, node2)
node4 = tf.constant(2.0, tf.float32)
node5 = tf.pow(node3, node4)
node6 = tf.constant(5.0, tf.float32)
node7 = tf.multiply(node5, node6)

print "3 + 4 = ",sess.run(node3)
print "(3 + 4)^2 = ",sess.run(node5)
print "((3 + 4)^2) * 5 = ",sess.run(node7)

3 + 4 =  7.0
(3 + 4)^2 =  49.0
((3 + 4)^2) * 5 =  245.0


In [7]:
# Compute 'x+y'

In [8]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
adder = x + y

z1 = sess.run(adder, {x: 3, y: 4.5})
print(z1)

z2 = sess.run(adder, {x: [1,3], y: [2, 4]})
print(z2)

7.5
[ 3.  7.]


In [9]:
# Compute '||Ax-y||'

array([[ 1.,  2.],
       [ 3.,  4.]], dtype=float32)

In [32]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
A = tf.constant([[1., 2.], [3., 4.]])
y_ = A*x
diff = y - y_
err = tf.norm(diff)

# should pass x to 2D matrix shape
print "A*x = \n",sess.run(y_, {x: [[1., 2.]]})
print "|Ax-y| = \n",sess.run(err, {x: [[1., 2.]], y:[8., 7.]})

A*x = 
[[ 1.  4.]
 [ 3.  8.]]
|Ax-y| = 
9.16515


In [33]:
# Variables & Learning a linear model

In [35]:
W = tf.Variable([.3], tf.float32)  # Variable adds learnable parameters in graph
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [36]:
# without initializing variable, running graph causes error
print(sess.run(linear_model, {x:[1,2,3,4]}))

FailedPreconditionError: Attempting to use uninitialized value Variable_2
	 [[Node: Variable_2/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_2"], _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_2)]]
	 [[Node: add_2/_121 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_9_add_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Variable_2/read', defined at:
  File "/home/daniel/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/daniel/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/daniel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/daniel/anaconda2/lib/python2.7/site-packages/traitlets/config/appl

In [37]:
# run variable initializer op, before running the graph
init = tf.global_variables_initializer()
sess.run(init)

In [38]:
# Then, it works
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [50]:
# Define loss
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

print "loss : ",sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]})

loss :  23.66


In [47]:
# assign different value to the variables
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])   # These are optimal parameters

print "Before assigning, W : ",sess.run(W)[0], "b : ",sess.run(b)[0]
sess.run([fixW, fixb])
print "After assigning, W : ",sess.run(W)[0], "b : ",sess.run(b)[0]

print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

Before assigning, W :  -1.0 b :  1.0
After assigning, W :  -1.0 b :  1.0
0.0


In [49]:
# Go back to the initial value
sess.run(init)
print "Ater initializing again, W : ",sess.run(W)[0], "b : ",sess.run(b)[0]

Ater initializing again, W :  0.3 b :  -0.3


In [56]:
# Learn the model using optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})
    
print "Learned model parameters, \t W : ",sess.run(W)[0], "b : ",sess.run(b)[0]

Learned model parameters, 	 W :  -0.999999 b :  0.999997
